In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
img1 = cv2.imread('Cimg.jpg',1)  # queryImage
img2 = cv2.imread('img.jpg',1) # trainImage

orb = cv2.KAZE_create(4000)
# Initialize ORB detector
# orb = cv2.ORB_create(4000,fastThreshold=30)
# orb = cv2.ORB_create(1200)
# orb = cv2.FastFeatureDetector_create(1200)
# orb = cv2.xfeatures2d.BriefDescriptorExtractor_create(1200)
print(orb)
# Find the keypoints and descriptors with ORB
kp1, des1 = orb.detectAndCompute(img1,None)
kp2, des2 = orb.detectAndCompute(img2,None)

index_params= dict(algorithm = 6, table_number = 6, key_size = 12, multi_probe_level = 1)

search_params = dict(checks=100)   

flann = cv2.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(des1,des2,k=2)

good_points = []
for m,n in matches:
    if m.distance < 0.60*n.distance:
        good_points.append(m)

<KAZE 0000026F22819ED0>


error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-u4kjpz2z\opencv\modules\flann\src\miniflann.cpp:336: error: (-210:Unsupported format or combination of formats) in function 'cv::flann::buildIndex_'
> type=5
> 

In [13]:
# min number of matches to call it a match.
MIN_MATCH_COUNT = 10

# Check if there are enough matches to detrmine if the object is actually present in the trainImg.
if len(good_points) > MIN_MATCH_COUNT: 
    
    # We are grabbing the locations of all the good matched points from both the images and formating them in a list
    src_pts = []
    dst_pts = []
    
    # Remember the indexes (e.g m.queryIdx) are the indexes of only the good matches in all the matches.
    for m in good_points:
        # This will be a tuple of x,y coordinate of a good match
        src_pts.append( kp1[m.queryIdx].pt )
        dst_pts.append( kp2[m.trainIdx].pt )
        
    # Converting to Float and Reshaping from shape (no_of_matches, 2) to (no_of_matches, 1, 2)    
    src_pts = np.float32(src_pts).reshape(-1, 1, 2) 
    dst_pts = np.float32(dst_pts).reshape(-1, 1, 2)

    # A 3x3 homography transformation matrix is obtained
    M , _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
       
    # Extracting the height and width of the Query Image.    
    h,w = img1.shape[:2]
    
    # This is the full size of our original Query Image.
    pts = np.float32([[0,0],[0,h],[w,h],[w,0] ]).reshape(-1,1,2)  
    
    # Takes in the initial points and transformation matrix and returns an array of same size as pts.
    # If your object is a rectangle then points in this array is just your usual 4 boundary points of that object.
    dst = cv2.perspectiveTransform(pts, M)
    
    # Drawing a polygon on the detected Object. We are not drawing a rectangle since your object may be rotated at an angle.
    final_image = cv2.polylines(img2.copy(), [np.int32(dst)], True, (255,0,0), 3, cv2.LINE_AA)  
else:
    print("Not enough matches are found " +str(len(good_points)))
plt.figure(figsize=[25,25])
plt.subplot(221);plt.imshow(img1[...,::-1]);plt.title("Query Img");plt.axis('off');
plt.subplot(222);plt.imshow(img2[...,::-1]);plt.title("Train Img");plt.axis('off');
plt.subplot(223);plt.imshow(final_image[...,::-1]);plt.title("Detected Object");plt.axis('off');

IndexError: list index out of range